In [1]:
import numpy as np
import torch
import dgl
import time

/home/bear/miniconda3/envs/dgl_v09/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 首先每个训练点有一个自己的标签(idx)
# 随后对标签进行传递 dst <-> src
# 在经过n轮之后,每个点都有了一个自己的最终标签
# 之后对聚类结果进行合并

graph = torch.as_tensor(np.fromfile("/raid/bear/data/raw/papers100M/graph1.bin",dtype=np.int32))
trainids = torch.as_tensor(np.fromfile("/raid/bear/data/raw/papers100M/trainIds.bin",dtype=np.int64))
src = graph[::2]
dst = graph[1::2]

In [3]:
src = src.cuda()
dst = dst.cuda()

In [4]:
nodeNUM = 111059956
nodeLabel = torch.zeros(nodeNUM).to(torch.int32) -1
nodeLabel[trainids] = trainids.to(torch.int32)
nodeLabel = nodeLabel.cuda()

In [7]:
print("nodeLabel :",nodeLabel)
dgl.lpGraph(src,dst,nodeLabel)
print("nodeLabel :",nodeLabel)

nodeLabel : tensor([    -1,     -1, 216539,  ...,  17929,     -1,     -1], device='cuda:0',
       dtype=torch.int32)
nodeLabel : tensor([ 2205,  9067, 16362,  ...,  1384,    -1,    -1], device='cuda:0',
       dtype=torch.int32)


In [8]:
src = src.cpu()
dst = dst.cpu()
nodeLabel = nodeLabel.cpu()
import gc
torch.cuda.empty_cache()
gc.collect()

55

In [9]:
# idx = torch.nonzero(nodeLabel >= 0).reshape(-1)
value = nodeLabel[trainids.to(torch.int64)]

In [17]:
binAns = torch.bincount(value)
torch.max(binAns)

tensor(687290)

In [18]:
s_binAns,_ = torch.sort(binAns,descending=True)

In [20]:
s_binAns[:40]

tensor([687290, 172884, 151759,  49527,  48329,  25771,   8999,   6149,   5545,
          4519,   3395,   3260,   3132,   2971,   2755,   2678,   2361,   2266,
          1972,   1824,   1733,   1442,   1391,   1320,   1075,   1072,    935,
           777,    735,    601,    594,    566,    534,    379,    337,    327,
           324,    306,    286,    278])

In [15]:
out = torch.nonzero(s_binAns > 30).reshape(-1)

In [16]:
out.shape

torch.Size([68])